[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/casangi/astrohack/blob/main/docs/tutorial_vla.ipynb)

![astrohack](_media/astrohack_logo.png)

In [1]:
import os

from importlib.metadata import version

try:
    import astrohack
    
    print('AstroHACK version', version('astrohack'), 'already installed.')
except ImportError as e:
    print(e)
    print('Installing AstroHACK')
    
    os.system("pip install astrohack")
    
    import astrohack 
    print('astrohack version', version('astrohack'), ' installed.')

AstroHACK version 0.2.2 already installed.


Run CASA script yada yada yada

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

client = astrohack_local_client(cores=2, memory_limit='8GB')
client

In [ ]:
# For the moment use my cal table, definetely not the data to put online
cal_table = 'Le cal table messieur'

locit_mds = extract_locit(cal_table, 
                          locit_name=None, 
                          ant_id=None, 
                          ddi=None, 
                          overwrite=False)

In [ ]:
locit_mds.summary()

In [ ]:
locit_mds.print_source_table()

In [ ]:
locit_mds.print_antenna_table()

In [ ]:
locit_plot_folder = 'locit_mds_plots'

In [ ]:
locit_mds.plot_source_positions(locit_plot_folder, 
                                display_labels=True, 
                                precessed=False)

In [ ]:
locit_mds.plot_antenna_positions(locit_plot_folder, 
                                 display_stations=True)

Now we proceed to actual fit yada yada yada

In [ ]:
position_mds = locit(locit_name, 
                     position_name=None, 
                     elevation_limit=10.0, 
                     polarization='both', 
                     fit_engine='linear algebra',
                     fit_kterm=False, 
                     fit_slope=True, 
                     ant_id=None, 
                     ddi=None, 
                     parallel=False, 
                     overwrite=False)

In [ ]:
position_mds.summary()

In [ ]:
position_export_folder = 'position_mds_export'

In [ ]:
position_mds.export_fit_results(position_export_folder, 
                                combine_ddis=False, 
                                position_unit='m', 
                                angle_unit='deg',
                                time_unit='hour')

In [ ]:
position_plot_folder = 'position_mds_plots'

In [ ]:
position_mds.plot_sky_coverage(position_plot_folder, 
                               ant_id=None,
                               ddi=None,
                               time_unit='hour', 
                               angle_unit='deg', 
                               display=True,
                               figure_size=None,
                               dpi=300,
                               parallel=False)

In [ ]:
position_mds.plot_gains(position_plot_folder, 
                        ant_id=None,
                        ddi=None,
                        time_unit='hour',
                        angle_unit='deg',
                        plot_fit=True,
                        display=True,
                        figure_size=None,
                        dpi=300,
                        parallel=False)